!pip install opendatasets
!pip install keras
!pip install tensorflow
!pip install rasa==1.1.4

import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops

import opendatasets as od
od.download("https://www.kaggle.com/datasets/koryakinp/fingers")

!pip install opencv-python
import cv2
import os



images=[]
labels=[]
dataset_dir='./fingers/test'
for filename in os.listdir(dataset_dir):
        image=cv2.imread(os.path.join(dataset_dir,filename))
        image=cv2.resize(image,(64,64))
        images=np.append(images,image)
        label=filename[-6]
        labels=np.append(labels,label)
        images=np.array(images)
        labels=np.array(labels)
        

        
        
        



def convolutional_model(input_shape):
    input_img=tf.keras.Input(shape=input_shape)
    x=tf.keras.layers.Conv2D(8,(4,4),strides=(1,1),padding='SAME')(input_img)
    x=tf.keras.layers.Activation('relu')(x)
    x=tf.keras.layers.MaxPooling2D((8,8),strides=(8,8),padding='SAME')(x)
    x=tf.keras.layers.Conv2D(16,(2,2),strides=(1,1),padding='SAME')(x)
    x=tf.keras.layers.Activation('relu')(x)
    x=tf.keras.layers.MaxPooling2D((4,4),strides=(4,4),padding='SAME')(x)
    x=tf.keras.layers.Flatten()(x)
    x=tf.keras.layers.Dense(6,activation='softmax')(x)
    model=tf.keras.Model(input_img,x)
    return model

In [11]:
model=convolutional_model((64,64,1))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 8)         136       
                                                                 
 activation (Activation)     (None, 64, 64, 8)         0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 8, 8, 8)           0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          528       
                                                                 
 activation_1 (Activation)   (None, 8, 8, 16)          0         
                                                          

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(images[:int(len(labels))],labels[:int (len(labels))],test_size=1/9,random_state=12)

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
x_train = x_train[:, 0,]
x_test=x_test[:,0,]
print(x_train.shape)
print(x_test.shape)

x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)
x_train = x_train[:, :,:,:,0,]
x_test=x_test[:,:,:,0]
y_train = y_train[:, :,:,:,0,]
y_test=y_test[:,:,:,0]
print(x_train.shape)
print(x_test.shape)
print ("y_train shape: " + str(y_train.shape))
print ("y_test shape: " + str(y_test.shape))

train_dataset = tf.data.Dataset.from_tensor_slices((x_train[1800], y_train[1800]))
train_dataset = train_dataset.batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test[300], y_test[300]))
test_dataset = test_dataset.batch(32)

history = model.fit(train_dataset,epochs=10, validation_data=test_dataset)

import random
import cv2
from matplotlib import pyplot as plt

numbers = list(range(1, 400))
random_array = random.sample(numbers, 10)

fig = plt.figure(figsize=(10, 7))

for i, idx in enumerate(random_array):
    img_resized = cv2.resize(x_test[idx], (64, 64))
    y_pred = np.argmax(model.predict(np.expand_dims(x_test[idx], axis=0),verbose=0))
    ax = fig.add_subplot(1, 10, i+1)
    ax.imshow(img_resized, cmap='gray')
    ax.axis('off')
    ax.set_title("pred = " + str(y_pred))

plt.tight_layout()
plt.show()